In [ ]:
import numpy as np
import copy
import os
os.chdir('..')
from dataloader import *
from models.vgg import VGG11
import argparse
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd.gradcheck import zero_gradients
from torch.autograd import Variable

In [13]:
class adversarial_attack():
    
    def __init__(self, method, model, dataloader, device, num_classes=10, save_data=False):
        self.method = method
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.num_classes = num_classes
        self.save_data = save_data
        
    def batch_attack(self):
        '''
        Run attack on a batch of data
        
        '''
        # Accuracy counter
        correct = 0
        total = 0
        adv_examples = []
        ct_save = 0
        adv_cat = torch.tensor([])

        # Loop over all examples in test set
        for data, label in self.dataloader:
            data, label = data.to(self.device), label.to(self.device)

            # Forward pass the data through the model
            output, _ = self.model(data)
            init_pred = output.max(1, keepdim=True)[1]  # get the index of the max log-probability

            if init_pred.item() != label.item():  # initially was incorrect --> no need to generate adversary
                continue

            # Call Attack
            if self.method in ['fgsm', 'stepll']:
                criterion = nn.CrossEntropyLoss()
                perturbed_data = self._FGSM(data, label, criterion, epsilon=0.05, clip_min=0.0, clip_max=1.0)
                
            elif self.method == 'jsma':
                # randomly select a target class
                target_class = init_pred
                while target_class == init_pred:
                    target_class = torch.randint(0, output.size()[1], (1,)).to(self.device)
                perturbed_data = self._JSMA(data, target_class, max_iter=100, clip_min=0.0, clip_max=1.0)
                
            elif self.method == 'deepfool':
                f_image = output.detach().cpu().numpy().flatten()
                I = (np.array(f_image)).flatten().argsort()[::-1]
                perturbed_data = self._deep_fool(data, label, I, overshoot=0.02, max_iter=100)
                
            elif self.method == 'cw':
                # randomly select a target class
                target_class = init_pred
                while target_class == init_pred:
                    target_class = torch.randint(0, output.size()[1], (1,)).to(self.device)
                perturbed_data = self._cw(data, target_class, max_steps=1000, clip_min=0.0, clip_max=1.0)
                
            else:
                print('Attack method is not supported')
                
            # Re-classify the perturbed image
            self.model.eval()
            with torch.no_grad():
                output, _ = self.model(perturbed_data)

            # Check for success
            final_pred = output.max(1, keepdim=True)[1]
            if final_pred.item() == label.item():
                correct += 1  # still correct
            else:# save successful attack
                if ct_save < 100:
                    if self.save_data:
                        os.makedirs('./data/normal_{}'.format(self.method), exist_ok=True)
                        os.makedirs('./data/adversarial_{}'.format(self.method), exist_ok=True)
                        # Save the original instance
                        torch.save((data.detach().cpu(), init_pred.detach().cpu()),
                                   './data/normal_{}/{}.pt'.format(self.method, ct_save))
                        # Save the adversarial example
                        torch.save((perturbed_data.detach().cpu(), final_pred.detach().cpu()),
                                   './data/adversarial_{}/{}.pt'.format(self.method, ct_save))
                ct_save += 1


#             else:  # attack is successful, final class is cat
#                 if final_pred.item() == 3:
#                     adv_cat = torch.cat([adv_cat, perturbed_data.detach().cpu()], dim=0)
#                     torch.save(adv_cat, './data/adv_cat_%s.pt'%self.method)

            adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
            adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
            total += 1

        # Calculate final accuracy
        final_acc = correct / float(len(self.dataloader))
        print("Test Accuracy = {} / {} = {}".format(correct, total, final_acc))

        # Return the accuracy and an adversarial example
        return final_acc, adv_examples
        
        
        
    def _FGSM(self, image, label, criterion, epsilon=0.05, clip_min=0.0, clip_max=1.0):
        ''' https://pytorch.org/tutorials/beginner/fgsm_tutorial.html
        FGSM attack
        Parameters:
            image: input image
            label: gt label
            criterion: loss function to use
            epsilon: perturbation strength
            clip_min, clip_max: minimum/maximum value a pixel can take
        '''
        
        pert_image = copy.deepcopy(image)
        x = Variable(pert_image, requires_grad=True)

        output, _ = self.model(x)

        if self.method == 'fgsm':
            loss = criterion(output, label)  # loss for ground-truth class
        else:
            ll = output.min(1, keepdim=True)[1][0]
            loss = criterion(output, ll)  # Loss for least-likely class

        # Back propogation
        self.model.zero_grad()
        loss.backward()

        # Collect the sign of the data gradient
        sign_data_grad = torch.sign(x.grad.data)

        # Create the perturbed image by adjusting each pixel of the input image
        if self.method == 'fgsm':
            perturbed_image = image + epsilon * sign_data_grad
        else:
            perturbed_image = image - epsilon * sign_data_grad

        # Adding clipping to maintain [0,1] range
        perturbed_image = torch.clamp(perturbed_image, clip_min, clip_max)

        return perturbed_image
        
    def _JSMA(self, image, target, max_iter=100, clip_min=0.0, clip_max=1.0):
        '''https://github.com/ast0414/adversarial-example/blob/master/craft.py
        Saliency map attack
        Parameters:
            image: input image
            target: target class
            max_iter: maximum iteration
            clip_min: minimum value of pixel
            clip_max: maximum value of pixel
        Returns:
            perturbed image
        '''
        
        def compute_jacobian(inputs, output):
            '''Helper function: compute jacobian matrix of confidence score vector w.r.t. input'''

            jacobian = torch.zeros(self.num_classes, *inputs.size()).cuda()

            for i in range(self.num_classes):
                zero_gradients(inputs)
                output[0, i].backward(retain_graph=True)
                jacobian[i] = inputs.grad.data

            return torch.transpose(jacobian, dim0=0, dim1=1)


        def saliency_map(jacobian, search_space, target_index):
            '''Helper function: compute saliency map and select the maximum index'''
            jacobian = jacobian.squeeze(0)
            alpha = jacobian[target_index]
            beta = jacobian.sum(0) - alpha

            mask1 = torch.ge(alpha, 0.0)
            mask2 = torch.le(beta, 0.0)

            mask = torch.mul(torch.mul(mask1, mask2), search_space)

            saliency_map = torch.mul(torch.mul(alpha, torch.abs(beta)), mask.float())
            saliency_map = saliency_map.sum(0).sum(0)

            row_idx, col_idx = (saliency_map == torch.max(saliency_map)).nonzero()[0]
            return row_idx, col_idx


        # Make a clone since we will alter the values
        pert_image = copy.deepcopy(image)
        x = Variable(pert_image, requires_grad=True)

        output, _ = self.model(x)
        _, label = torch.max(output.data, 1)
        
        count = 0
        # if attack is successful or reach the maximum number of iterations
        while (count < max_iter) and (label != target):

            search_space = (x.data[0] > clip_min) & (x.data[0] < clip_max)

            # Calculate Jacobian
            jacobian = compute_jacobian(x, output)

            # get the highest saliency map's index
            row_idx, col_idx = saliency_map(jacobian, search_space, target)

            # increase to its maximum value
            x.data[0, :, row_idx, col_idx] = clip_max

            # recompute prediction
            output, _ = self.model(x)
            label = output.max(1, keepdim=True)[1]

            count += 1

        return x.data
    
    
    
    def _deep_fool(self, image, label, I, overshoot=0.02, max_iter=100):
        '''https://github.com/LTS4/DeepFool/tree/master/Python
        DeepFool attack
        Parameters:
            image: input image
            label: ground-truth label
            I: current predicted class ranked by decending order
            overshoot: scale factor to increase perturbation a little bit
            max_iter: maximum iterations allowed
        Returns:
            perturbed image
        '''

        pert_image = copy.deepcopy(image)
        w = np.zeros(image.shape)
        r_tot = np.zeros(image.shape)

        loop_i = 0

        x = Variable(pert_image, requires_grad=True)
        fs, _ = self.model(x)
        fs_list = [fs[0, I[k]] for k in range(self.num_classes)]
        k_i = label

        # Stop until attack is successful or reach the maximum iterations
        while k_i == label and loop_i < max_iter:

            pert = np.inf
            fs[0, I[0]].backward(retain_graph=True) # backpropogate the maximum confidence score
            grad_orig = x.grad.data.detach().cpu().numpy().copy()

            for k in range(1, self.num_classes):
                zero_gradients(x)

                fs[0, I[k]].backward(retain_graph=True)
                cur_grad = x.grad.data.detach().cpu().numpy().copy()

                # set new w_k and new f_k
                w_k = cur_grad - grad_orig
                f_k = (fs[0, I[k]] - fs[0, I[0]]).data.detach().cpu().numpy()

                pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())

                # determine which w_k to use
                if pert_k < pert:
                    pert = pert_k
                    w = w_k

            # compute r_i and r_tot
            # Added 1e-4 for numerical stability
            r_i =  (pert+1e-4) * w / np.linalg.norm(w)
            r_tot = np.float32(r_tot + r_i)

            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()

            x = Variable(pert_image, requires_grad=True)
            fs, _ = self.model(x)
            k_i = np.argmax(fs.data.detach().cpu().numpy().flatten())

            loop_i += 1

        return x.data
    
    def _cw(self, image, target, max_steps=1000, clip_min=0.0, clip_max=1.0):
        '''https://github.com/rwightman/pytorch-nips2017-attack-example/blob/master/attacks/attack_carlini_wagner_l2.py
           C&W L2 attack
           Parameters:
               image: input image
               target: adv target class
               max_steps: maximum iterations of optimization
               clip_min, clip_max: clip image into legal range
        '''
        
        confidence = 20  # FIXME need to find a good value for this, 0 value used in paper not doing much...
        initial_const = 0.1  # bumped up from default of .01 in reference code
        binary_search_steps = 5
        repeat = binary_search_steps >= 10
        abort_early = True
        debug = False
        batch_idx = 0

        def reduce_sum(x, keepdim=True):
            '''Helper function
               Perform sum on all dimension except for batch dim
            '''
            for a in reversed(range(1, x.dim())):
                x = x.sum(a, keepdim=keepdim)
            return x

        def l2_dist(x, y, keepdim=True):
            '''Helper function
               Compute L2-dist
            '''
            d = (x - y)**2
            return reduce_sum(d, keepdim=keepdim)

        def torch_arctanh(x, eps=1e-6):
            '''Helper function
               Implement arctanh function
            '''
            x *= (1. - eps)
            return (torch.log((1 + x) / (1 - x))) * 0.5

        def tanh_rescale(x, x_min=0., x_max=1.):
            '''Helper function
               Implement tanh function
            '''
            return (torch.tanh(x) + 1) * 0.5 * (x_max - x_min) + x_min

        def compare(output, target):
            '''Helper function
               Compare predicted value with ground-truth
            '''
            if not isinstance(output, (float, int, np.int64)):
                output = np.copy(output)
                output[target] -= confidence
                output = np.argmax(output)
            return output == target

        def cal_loss(output, target, dist, scale_const):
            '''Helper function
               Compute loss for C&W L2
            '''
            # compute the probability of the label class versus the maximum other
            real = (target * output).sum(1)
            other = ((1. - target) * output - target * 10000.).max(1)[0]
            # if targeted, optimize for making the other class most likely
            loss1 = torch.clamp(other - real + confidence, min=0.)  # equiv to max(..., 0.)
            loss1 = torch.sum(scale_const * loss1)

            loss2 = dist.sum()

            loss = loss1 + loss2
            return loss


        def optimize(optimizer, input_var, modifier_var, target_var, scale_const_var, input_orig=None):
            '''Helper function
               Optimize C&W L2 loss by Adam
            '''
            # apply modifier and clamp resulting image to keep bounded from clip_min to clip_max
            input_adv = tanh_rescale(modifier_var + input_var, clip_min, clip_max)

            output, _ = self.model(input_adv)

            # distance to the original input data
            if input_orig is None:
                dist = l2_dist(input_adv, input_var, keepdim=False)
            else:
                dist = l2_dist(input_adv, input_orig, keepdim=False)

            loss = cal_loss(output, target_var, dist, scale_const_var)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_np = loss.item()
            dist_np = dist.data.detach().cpu().numpy()
            output_np = output.data.detach().cpu().numpy()
            input_adv_np = input_adv.data # back to BHWC for numpy consumption
            return loss_np, dist_np, output_np, input_adv_np

        batch_size = image.size(0)

        # set the lower and upper bounds accordingly
        lower_bound = np.zeros(batch_size)
        scale_const = np.ones(batch_size) * initial_const
        upper_bound = np.ones(batch_size) * 1e10

        # python/numpy placeholders for the overall best l2, label score, and adversarial image
        o_best_l2 = [1e10] * batch_size
        o_best_score = [-1] * batch_size
        o_best_attack = image
        
        # setup input (image) variable, clamp/scale as necessary
        # convert to tanh-space, input already int -1 to 1 range, does it make sense to do
        # this as per the reference implementation or can we skip the arctanh?
        input_var = Variable(torch_arctanh(image), requires_grad=False)
        input_orig = tanh_rescale(input_var, clip_min, clip_max)

        # setup the target variable, we need it to be in one-hot form for the loss function
        target_onehot = torch.zeros(target.size() + (self.num_classes,)).to(self.device)
        target_onehot.scatter_(1, target.unsqueeze(1), 1.)
        target_var = Variable(target_onehot, requires_grad=False)

        # setup the modifier variable, this is the variable we are optimizing over
        modifier = torch.zeros(input_var.size()).float()
        modifier = modifier.to(self.device)
        modifier_var = Variable(modifier, requires_grad=True)

        optimizer = optim.Adam([modifier_var], lr=0.0005)

        for search_step in range(binary_search_steps):
            print('Batch: {0:>3}, search step: {1}'.format(batch_idx, search_step))
            if debug:
                print('Const:')
                for i, x in enumerate(scale_const):
                    print(i, x)
            best_l2 = [1e10] * batch_size
            best_score = [-1] * batch_size

            # The last iteration (if we run many steps) repeat the search once.
            if repeat and search_step == binary_search_steps - 1:
                scale_const = upper_bound

            scale_const_tensor = torch.from_numpy(scale_const).float()
            scale_const_tensor = scale_const_tensor.to(self.device)
            scale_const_var = Variable(scale_const_tensor, requires_grad=False)

            prev_loss = 1e6
            for step in range(max_steps):
                # perform the attack
                loss, dist, output, adv_img = optimize(
                    optimizer,
                    input_var,
                    modifier_var,
                    target_var,
                    scale_const_var,
                    input_orig)

                if step % 100 == 0 or step == max_steps - 1:
                    print('Step: {0:>4}, loss: {1:6.4f}, dist: {2:8.5f}, modifier mean: {3:.5e}'.format(
                        step, loss, dist.mean(), modifier_var.data.mean()))

                if abort_early and step % (max_steps // 10) == 0:
                    if loss > prev_loss * .9999:
                        print('Aborting early...')
                        break
                    prev_loss = loss

                # update best result found
                for i in range(batch_size):
                    target_label = target[i]
                    output_logits = output[i]
                    output_label = np.argmax(output_logits)
                    di = dist[i]
                    if debug:
                        if step % 100 == 0:
                            print('{0:>2} dist: {1:.5f}, output: {2:>3}, {3:5.3}, target {4:>3}'.format(
                                i, di, output_label, output_logits[output_label], target_label))
                    if di < best_l2[i] and compare(output_logits, target_label):
                        if debug:
                            print('{0:>2} best step,  prev dist: {1:.5f}, new dist: {2:.5f}'.format(
                                  i, best_l2[i], di))
                        best_l2[i] = di
                        best_score[i] = output_label
                    if di < o_best_l2[i] and compare(output_logits, target_label):
                        if debug:
                            print('{0:>2} best total, prev dist: {1:.5f}, new dist: {2:.5f}'.format(
                                  i, o_best_l2[i], di))
                        o_best_l2[i] = di
                        o_best_score[i] = output_label
                        o_best_attack[i] = adv_img[i]

                sys.stdout.flush()
                # end inner step loop

            # adjust the constants
            batch_failure = 0
            batch_success = 0
            for i in range(batch_size):
                if compare(best_score[i], target[i]) and best_score[i] != -1:
                    # successful, do binary search and divide const by two
                    upper_bound[i] = min(upper_bound[i], scale_const[i])
                    if upper_bound[i] < 1e9:
                        scale_const[i] = (lower_bound[i] + upper_bound[i]) / 2
                    if debug:
                        print('{0:>2} successful attack, lowering const to {1:.3f}'.format(
                            i, scale_const[i]))
                else:
                    # failure, multiply by 10 if no solution found
                    # or do binary search with the known upper bound
                    lower_bound[i] = max(lower_bound[i], scale_const[i])
                    if upper_bound[i] < 1e9:
                        scale_const[i] = (lower_bound[i] + upper_bound[i]) / 2
                    else:
                        scale_const[i] *= 10
                    if debug:
                        print('{0:>2} failed attack, raising const to {1:.3f}'.format(
                            i, scale_const[i]))
                if compare(o_best_score[i], target[i]) and o_best_score[i] != -1:
                    batch_success += 1
                else:
                    batch_failure += 1

            print('Num failures: {0:2d}, num successes: {1:2d}\n'.format(batch_failure, batch_success))
            sys.stdout.flush()
            # end outer search loop

        return o_best_attack
    

In [2]:
# '''load cifar10 dataset'''
# trainloader, testloader, classes = load_cifar_data()

# '''define model'''
# print("CUDA Available: ", torch.cuda.is_available())
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# criterion = nn.CrossEntropyLoss()

# # Initialize the network
# model = VGG11()
# model.to(device)

# '''load saved model'''
# model.load_state_dict(torch.load('./checkpoints/model.th')['state_dict'])
# model.eval()

# device = 'cuda'
# check = adversarial_attack(method='jsma', model=model, dataloader=testloader, device=device, num_classes=10, save_data=True)

# check.batch_attack()

In [3]:
'''load cifar10 dataset'''
trainloader, testloader, classes = load_cifar_data()

'''define model'''
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Initialize the network
model = VGG11()
model.to(device)

'''load saved model'''
model.load_state_dict(torch.load('./checkpoints/model.th')['state_dict'])
model.eval()

device = 'cuda'
check = adversarial_attack(method='fgsm', model=model, dataloader=testloader, device=device, num_classes=10, save_data=True)

check.batch_attack()

Files already downloaded and verified
Files already downloaded and verified
CUDA Available:  True
Test Accuracy = 193 / 7867 = 0.0193


(0.0193,
 [(8, 1, array([[[0.8715686 , 0.9558824 , 0.8598039 , ..., 0.8637255 ,
            0.8637255 , 0.95980394],
           [0.8833333 , 0.8715686 , 0.8715686 , ..., 0.9754902 ,
            0.9754902 , 0.97156864],
           [0.87941176, 0.86764705, 0.86764705, ..., 0.97156864,
            0.8715686 , 0.86764705],
           ...,
           [0.29117647, 0.21862745, 0.12450981, ..., 0.6127451 ,
            0.7637255 , 0.7872549 ],
           [0.27156863, 0.23039216, 0.19117647, ..., 0.732353  ,
            0.7754902 , 0.68333334],
           [0.38333336, 0.19313726, 0.27745098, ..., 0.60882354,
            0.65588236, 0.7794118 ]],
   
          [[0.97156864, 0.9558824 , 0.8598039 , ..., 0.9637255 ,
            0.9637255 , 0.8598039 ],
           [0.8833333 , 0.8715686 , 0.97156864, ..., 0.9754902 ,
            0.8754902 , 0.97156864],
           [0.9794118 , 0.9676471 , 0.86764705, ..., 0.97156864,
            0.97156864, 0.9676471 ],
           ...,
           [0.4382353 , 0.25  

In [ ]:
'''load cifar10 dataset'''
trainloader, testloader, classes = load_cifar_data()

'''define model'''
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Initialize the network
model = VGG11()
model.to(device)

'''load saved model'''
model.load_state_dict(torch.load('./checkpoints/model.th')['state_dict'])
model.eval()

device = 'cuda'
check = adversarial_attack(method='stepll', model=model, dataloader=testloader, device=device, num_classes=10, save_data=True)

check.batch_attack()

In [ ]:
'''load cifar10 dataset'''
trainloader, testloader, classes = load_cifar_data()

'''define model'''
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Initialize the network
model = VGG11()
model.to(device)

'''load saved model'''
model.load_state_dict(torch.load('./checkpoints/model.th')['state_dict'])
model.eval()

device = 'cuda'
check = adversarial_attack(method='deepfool', model=model, dataloader=testloader, device=device, num_classes=10, save_data=True)

check.batch_attack()

In [14]:
'''load cifar10 dataset'''
trainloader, testloader, classes = load_cifar_data()

'''define model'''
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Initialize the network
model = VGG11()
model.to(device)

'''load saved model'''
model.load_state_dict(torch.load('./checkpoints/model.th')['state_dict'])
model.eval()

device = 'cuda'
check = adversarial_attack(method='cw', model=model, dataloader=testloader, device=device, num_classes=10, save_data=True)

acc, ex = check.batch_attack()

Step:  200, loss: 0.1641, dist:  0.16339, modifier mean: -4.13860e-05
Step:  300, loss: 0.1646, dist:  0.16459, modifier mean: -4.15810e-05
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 2
Step:    0, loss: 0.1644, dist:  0.16435, modifier mean: -4.12605e-05
Step:  100, loss: 0.1612, dist:  0.14936, modifier mean: -4.01222e-05
Step:  200, loss: 0.1605, dist:  0.14571, modifier mean: -4.49817e-05
Step:  300, loss: 0.1604, dist:  0.14382, modifier mean: -5.01036e-05
Step:  400, loss: 0.1601, dist:  0.14313, modifier mean: -5.07397e-05
Step:  500, loss: 0.1598, dist:  0.14291, modifier mean: -5.25042e-05
Step:  600, loss: 0.1605, dist:  0.14230, modifier mean: -5.24428e-05
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 3
Step:    0, loss: 0.1515, dist:  0.14226, modifier mean: -5.22167e-05
Step:  100, loss: 0.1294, dist:  0.08173, modifier mean: -3.29185e-05
Step:  200, loss: 0.1264, dist:  0.07151, modifier mean: -3.82094e

Step:  999, loss: 0.6718, dist:  0.57342, modifier mean: -6.60548e-04
Num failures:  1, num successes:  0

Batch:   0, search step: 1
Step:    0, loss: 1.5524, dist:  0.57358, modifier mean: -6.66568e-04
Step:  100, loss: 0.7061, dist:  0.70606, modifier mean: -7.26005e-04
Step:  200, loss: 0.6921, dist:  0.69213, modifier mean: -6.98986e-04
Step:  300, loss: 0.6841, dist:  0.68407, modifier mean: -6.76451e-04
Step:  400, loss: 0.6617, dist:  0.66168, modifier mean: -6.42088e-04
Step:  500, loss: 0.6669, dist:  0.66694, modifier mean: -6.61249e-04
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 2
Step:    0, loss: 0.6664, dist:  0.66637, modifier mean: -6.60758e-04
Step:  100, loss: 0.6438, dist:  0.63800, modifier mean: -6.49394e-04
Step:  200, loss: 0.6408, dist:  0.63171, modifier mean: -6.55300e-04
Step:  300, loss: 0.6354, dist:  0.63545, modifier mean: -6.57707e-04
Step:  400, loss: 0.6362, dist:  0.63621, modifier mean: -6.64902e-04
Aborting early

Step:  300, loss: 0.2898, dist:  0.22951, modifier mean: -6.38343e-04
Step:  400, loss: 0.2893, dist:  0.22652, modifier mean: -6.40040e-04
Step:  500, loss: 0.2886, dist:  0.22472, modifier mean: -6.43604e-04
Step:  600, loss: 0.2883, dist:  0.22377, modifier mean: -6.49953e-04
Step:  700, loss: 0.2882, dist:  0.22333, modifier mean: -6.51856e-04
Step:  800, loss: 0.2878, dist:  0.22296, modifier mean: -6.49154e-04
Step:  900, loss: 0.2881, dist:  0.22262, modifier mean: -6.47392e-04
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 3
Step:    0, loss: 0.3206, dist:  0.22261, modifier mean: -6.47686e-04
Step:  100, loss: 0.3070, dist:  0.27718, modifier mean: -7.41885e-04
Step:  200, loss: 0.3051, dist:  0.28691, modifier mean: -7.28620e-04
Step:  300, loss: 0.3049, dist:  0.28971, modifier mean: -7.25288e-04
Step:  400, loss: 0.3042, dist:  0.29087, modifier mean: -7.17001e-04
Step:  500, loss: 0.3046, dist:  0.29144, modifier mean: -7.08664e-04
Aborting

Step:  999, loss: 1.1962, dist:  1.18799, modifier mean: -5.73868e-04
Num failures:  0, num successes:  1

Batch:   0, search step: 2
Step:    0, loss: 1.1894, dist:  1.18938, modifier mean: -5.75213e-04
Step:  100, loss: 1.1663, dist:  1.15674, modifier mean: -5.17100e-04
Step:  200, loss: 1.1574, dist:  1.15736, modifier mean: -5.11448e-04
Step:  300, loss: 1.1561, dist:  1.14636, modifier mean: -4.97756e-04
Step:  400, loss: 1.1509, dist:  1.14728, modifier mean: -4.85214e-04
Step:  500, loss: 1.1446, dist:  1.14458, modifier mean: -4.85102e-04
Step:  600, loss: 1.1449, dist:  1.13812, modifier mean: -4.77628e-04
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 3
Step:    0, loss: 1.1395, dist:  1.13949, modifier mean: -4.78702e-04
Step:  100, loss: 1.1306, dist:  1.13056, modifier mean: -4.69558e-04
Step:  200, loss: 1.1267, dist:  1.12673, modifier mean: -4.61545e-04
Step:  300, loss: 1.1272, dist:  1.12719, modifier mean: -4.61160e-04
Aborting early

Num failures:  0, num successes:  1

Batch:   0, search step: 0
Step:    0, loss: 2.6704, dist:  0.00000, modifier mean: 1.95318e-06
Step:  100, loss: 1.1224, dist:  0.22022, modifier mean: -4.72091e-05
Step:  200, loss: 0.7985, dist:  0.37614, modifier mean: -1.15808e-04
Step:  300, loss: 0.6707, dist:  0.46306, modifier mean: -1.05501e-04
Step:  400, loss: 0.6046, dist:  0.51636, modifier mean: -1.27033e-04
Step:  500, loss: 0.5692, dist:  0.54726, modifier mean: -1.24521e-04
Step:  600, loss: 0.5495, dist:  0.54678, modifier mean: -1.17578e-04
Step:  700, loss: 0.5396, dist:  0.53584, modifier mean: -7.93302e-05
Step:  800, loss: 0.5298, dist:  0.52958, modifier mean: -8.34135e-05
Step:  900, loss: 0.5265, dist:  0.52578, modifier mean: -9.36829e-05
Step:  999, loss: 0.5246, dist:  0.52141, modifier mean: -8.15261e-05
Num failures:  0, num successes:  1

Batch:   0, search step: 1
Step:    0, loss: 0.5235, dist:  0.52147, modifier mean: -8.12814e-05
Step:  100, loss: 0.4989, dist:  

...

Step:  600, loss: 0.5526, dist:  0.54244, modifier mean: 1.16405e-04
Step:  700, loss: 0.5519, dist:  0.54711, modifier mean: 1.36098e-04
Step:  800, loss: 0.5496, dist:  0.54821, modifier mean: 1.69937e-04
Step:  900, loss: 0.5484, dist:  0.54737, modifier mean: 1.93509e-04
Step:  999, loss: 0.5479, dist:  0.54523, modifier mean: 2.10190e-04
Num failures:  0, num successes:  1

Batch:   0, search step: 3
Step:    0, loss: 0.5475, dist:  0.54477, modifier mean: 2.11113e-04
Step:  100, loss: 0.5433, dist:  0.51480, modifier mean: 3.15848e-04
Step:  200, loss: 0.5430, dist:  0.50648, modifier mean: 3.26064e-04
Step:  300, loss: 0.5420, dist:  0.50258, modifier mean: 3.26639e-04
Step:  400, loss: 0.5407, dist:  0.50108, modifier mean: 3.22194e-04
Step:  500, loss: 0.5404, dist:  0.49978, modifier mean: 3.14955e-04
Step:  600, loss: 0.5407, dist:  0.49889, modifier mean: 3.03319e-04
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 4
Step:    0, loss: 0.5450, 

Step:  100, loss: 0.2726, dist:  0.14081, modifier mean: -2.84863e-03
Step:  200, loss: 0.2725, dist:  0.14525, modifier mean: -3.13695e-03
Step:  300, loss: 0.2718, dist:  0.14683, modifier mean: -3.38392e-03
Step:  400, loss: 0.2717, dist:  0.14727, modifier mean: -3.62455e-03
Step:  500, loss: 0.2717, dist:  0.14730, modifier mean: -3.87100e-03
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 0
Step:    0, loss: 3.4734, dist:  0.00000, modifier mean: 1.36233e-05
Step:  100, loss: 1.8242, dist:  0.23202, modifier mean: 4.16677e-04
Step:  200, loss: 1.2637, dist:  0.47346, modifier mean: 3.76055e-04
Step:  300, loss: 0.9726, dist:  0.65122, modifier mean: -3.40163e-05
Step:  400, loss: 0.8279, dist:  0.74986, modifier mean: -4.79688e-04
Step:  500, loss: 0.7593, dist:  0.75626, modifier mean: -8.49300e-04
Step:  600, loss: 0.7195, dist:  0.71895, modifier mean: -1.05033e-03
Step:  700, loss: 0.7017, dist:  0.70107, modifier mean: -1.22527e-03
Step:  800,

Step:  900, loss: 0.2090, dist:  0.20896, modifier mean: -6.17207e-05
Step:  999, loss: 0.2077, dist:  0.20631, modifier mean: -5.56197e-05
Num failures:  0, num successes:  1

Batch:   0, search step: 1
Step:    0, loss: 0.2072, dist:  0.20724, modifier mean: -5.49488e-05
Step:  100, loss: 0.2055, dist:  0.20550, modifier mean: -4.16558e-05
Step:  200, loss: 0.2059, dist:  0.20503, modifier mean: -2.78151e-05
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 2
Step:    0, loss: 0.2052, dist:  0.20503, modifier mean: -2.79483e-05
Step:  100, loss: 0.2043, dist:  0.20139, modifier mean: -7.72604e-06
Step:  200, loss: 0.2041, dist:  0.20035, modifier mean: 1.59624e-05
Step:  300, loss: 0.2041, dist:  0.19996, modifier mean: 4.73087e-05
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 3
Step:    0, loss: 0.2019, dist:  0.19994, modifier mean: 4.76356e-05
Step:  100, loss: 0.1771, dist:  0.13112, modifier mean: 1.60586e-04
Step: 

Step:  500, loss: 0.2883, dist:  0.28831, modifier mean: -4.69848e-04
Step:  600, loss: 0.2820, dist:  0.28199, modifier mean: -5.28549e-04
Step:  700, loss: 0.2793, dist:  0.27927, modifier mean: -5.83698e-04
Step:  800, loss: 0.2790, dist:  0.27902, modifier mean: -6.31177e-04
Step:  900, loss: 0.2798, dist:  0.27541, modifier mean: -6.40634e-04
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 1
Step:    0, loss: 0.2755, dist:  0.27533, modifier mean: -6.41599e-04
Step:  100, loss: 0.2732, dist:  0.27253, modifier mean: -6.64841e-04
Step:  200, loss: 0.2725, dist:  0.27254, modifier mean: -6.85786e-04
Step:  300, loss: 0.2728, dist:  0.27226, modifier mean: -7.01636e-04
Aborting early...
Num failures:  0, num successes:  1

Batch:   0, search step: 2
Step:    0, loss: 0.2722, dist:  0.27221, modifier mean: -7.01210e-04
Step:  100, loss: 0.2704, dist:  0.26197, modifier mean: -7.01964e-04
Step:  200, loss: 0.2697, dist:  0.25833, modifier mean: -7.16124e

Step:  300, loss: 0.1847, dist:  0.13697, modifier mean: -8.47455e-05
Step:  400, loss: 0.1845, dist:  0.13747, modifier mean: -8.06151e-05
Step:  500, loss: 0.1847, dist:  0.13759, modifier mean: -7.60525e-05
Aborting early...
Num failures:  0, num successes:  1

Test Accuracy = 0 / 7867 = 0.0
